In [ ]:
import logging
import constants
import boto3 # had to downgrade boto3 to 1.35.99
from botocore.exceptions import ClientError
import os
import json
import shutil

#set up logging 
logging.basicConfig(level=logging.ERROR)
debug=logging.debug
info=logging.info
warning=logging.warning
error=logging.error

In [ ]:
# use third party object storage to create an S3 Client
s3_client = boto3.client(
    "s3",
    endpoint_url=constants.AWS_S3_ENDPOINT,
    aws_access_key_id=constants.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=constants.AWS_SECRET_ACCESS_KEY,
)

bucket = constants.AWS_S3_BUCKET

In [ ]:
def create_url(bucket_name: str,
            object_name: str):
 
    """
           
    This function takes a bucket name, an object name, and an expiration time (in seconds) and generates a URL download link for the object.

    Arguments:
        bucket_name: String of name of the bucket
        object_name: Name of the object (key) that the URL will be pointed to

    Returns:
        Link of output (object download) URL
        
    Raises: 
        Exceptions raised will display an error message and be logged in the export.log file
    """
    try:
        if r':443' in constants.AWS_S3_ENDPOINT:
            endpoint=constants.AWS_S3_ENDPOINT.split(':')
            endpoint=fr"{endpoint[0]}:{endpoint[1]}"
        else:
            endpoint=constants.AWS_S3_ENDPOINT
        response=os.path.join(endpoint,bucket_name,object_name)
    except ClientError as e:
        print(e)
        return None
    return response

def set_permissions(bucket_name: str,
                    object_name: str,
                    permissions='public-read'):
    """
    This function takes a bucket name, an object name, and a permissions value (specified below) and sets the object's permissions to the value given.

    Arguments:
        bucket_name: String of name of the bucket
        object_name: Name of the object (key) that the URL will be pointed to
        permissions: If not specified, the permissions will default to 'public-read'. Otherwise, permissions can be found below:
        'private'|'public-read'|'public-read-write'|'authenticated-read'|'aws-exec-read'|'bucket-owner-read'|'bucket-owner-full-control'

    Returns:
        Nothing
        
    Raises: 
        Exceptions raised will display an error message and be logged in the export.log file 
    """

    try:
        response = s3_client.put_object_acl(ACL=permissions, Bucket=bucket_name, Key=object_name)
        print(f'Set permissions on {object_name} success, set to {permissions}')
    except Exception as e:
        print(f'Error when setting permission: double check permission: {permissions}. Refer to help(set_permissions) for documentation.')
        print(e)

    return

In [ ]:
transfer=boto3.s3.transfer.S3Transfer(s3_client)

# List to store uploaded file URLs
uploaded_files = []

# import boto3    
# s3 = boto3.resource('s3')
# s3object = s3.Object('your-bucket-name', 'your_file.json')

# s3object.put(
#     Body=(bytes(json.dumps(json_data).encode('UTF-8')))
# )
def upload_folder(json_folder, s3_folder):
    for root, _, files in os.walk(json_folder):
        for file in files:
            local_file_path = os.path.join(root, file)

            if os.path.exists(local_file_path):
                print(f"Uploading {local_file_path} to {bucket}/{s3_key}")
                relative_path = os.path.relpath(local_file_path, json_folder)
                s3_key = os.path.join(s3_folder, relative_path).replace("\\", "/")

                try:
                    # s3_client.upload_file(local_file_path, bucket, s3_key)
                    # print(f"Uploaded: {s3_key}")
                    # print(f"Uploaded: {s3_key}")
                    transfer.upload_file(local_file_path, bucket, s3_key)
                    url=create_url(bucket, s3_key)
                    set_permissions(bucket, s3_key) # default is public-read
                    print(url)
                except Exception as e:
                    print(f"Failed to upload {file}: {e}")
                    
def upload_png(png_folder, s3_folder):
    if os.path.exists(png_folder):
        print(' folder')
    else: 
        print( 'no folder')
    for root, _, files in os.walk(png_folder):
        for file in files:
            print(file)
            local_file_path = os.path.join(root, file)
            
            if os.path.exists(local_file_path):
                print('file exists')
                file_name=file.split('.')[0]
                relative_path = os.path.relpath(local_file_path, png_folder)
                s3_key = os.path.join(s3_folder,file_name, relative_path).replace("\\", "/")
                print(s3_key)
                print(f"Uploading {local_file_path} to {bucket}/{s3_key}")

                # try:
                    # s3_client.upload_file(local_file_path, bucket, s3_key)
                    # print(f"Uploaded: {s3_key}")
                    # print(f"Uploaded: {s3_key}")
                transfer.upload_file(local_file_path, bucket, s3_key)
                url=create_url(bucket, s3_key)
                set_permissions(bucket, s3_key) # default is public-read
                print(url)
                # except Exception as e:
                #     print(f"Failed to upload {file}: {e}")
           
            

In [ ]:
# Configuration

FOLDER_PATH = r"\stac\output" # Replace with the path to your local folder
json_folder=r"jsons/"  
png_folder= os.path.join(FOLDER_PATH,"thumbnails")
S3_BASE_PATH = "BC_STAC"  # Folder name in S3

# Run upload function
# upload_folder(FOLDER_PATH, S3_BASE_PATH)
upload_png(png_folder,f"{S3_BASE_PATH}/DEM")